In [16]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import nibabel as nib

In [ ]:
mask_path = "/home/diogommiranda/tese/masks/ROI_MASK.nii"
img = nib.load(mask_path)
volume = img.get_fdata(dtype=np.float32)
current_min = np.min(volume)
current_max = np.max(volume)
print(f"Calculated Min: {current_min}, Max: {current_max}")
volume = np.transpose(volume, (2, 1, 0)) # (D, H, W)
volume = np.expand_dims(volume, axis=0) # Add channel dimension
volume = np.expand_dims(volume, axis=0) # Add batch dimension
volume = torch.from_numpy(volume)
print(f"Shape after transpose: {volume.shape}")

Calculated Min: 0.0, Max: 1.0
Shape after transpose: torch.Size([1, 1, 91, 109, 91])


In [71]:
roi_scores = F.interpolate(
    input = volume,
    mode = 'area',
    size = (6, 7, 6)
)
print(roi_scores.shape)
roi_scores = roi_scores.squeeze(0)
print(roi_scores.shape)

torch.Size([1, 1, 6, 7, 6])
torch.Size([1, 6, 7, 6])


In [72]:
roi_min = torch.min(roi_scores)
roi_max = torch.max(roi_scores)
print(f"ROI Min: {roi_min.item()}, ROI Max: {roi_max.item()}")

ROI Min: 0.0, ROI Max: 0.572021484375


In [76]:
token_roi_scores = roi_scores.flatten(start_dim=1)
print(f"Token ROI Scores Shape: {token_roi_scores.shape}")

Token ROI Scores Shape: torch.Size([1, 252])


In [77]:
token_roi_scores_softmax = F.softmax(token_roi_scores, dim=1)
print(f"Token ROI Scores Softmax Shape: {token_roi_scores_softmax.shape}")

Token ROI Scores Softmax Shape: torch.Size([1, 252])


In [79]:
softmax_min = torch.min(token_roi_scores_softmax)
softmax_max = torch.max(token_roi_scores_softmax)
print(f"Softmax Min: {softmax_min}, Softmax Max: {softmax_max}")

Softmax Min: 0.003775653662160039, Softmax Max: 0.006689873989671469
